In [1]:
import pandas as pd

PRODUCT = [
    "Sportsbook", "Number Game", "Virtual Sports", "Saba Casino", "RNG Keno", "AG Casino",
    "Sportsbook 2", "RNG Slot", "Cricket", "Allbet", "Macau Games", "Cash Out", "Lottery",
    "Voidbrige Jackpot", "Exchange", "PP", "Arcadia Gaming", "SG", "Saba Promotion",
    "Virtual Games", "SA Gaming", "Table Game", "Live Casino", "SABA.games", "Togel 4D",
    "RNG Games", "AE Sexy", "IBCBet Live Casino", "BBIN", "GPI", "WM", "ION", "RNG Casino",
    "Saba Virtual Sports", "PG Soft", "Joker", "BG", "MaxGame", "Habanero", "CG",
    "Sports Lottery", "PP Live Casino", "Vgaming", "AdvantPlay", "AdvantPlay Mini", 
    "Bitcoin", "FGG", "SEAL Entertainment", "MG-RNG", "Player Tips", "Jili", "YeeBet", 
    "UU SLOTS", "Live22", "WE Live Casino", "Saba Coins", "Yolo Play", "Nextspin",
    "SABA xD", "FastSpin", "FA CHAI", "GPI Live Casino", "PLAYSTAR", "HOTDOG", "ON Casino",
    "Smartsoft", "Playtech", "SABA Keno", "ASKMESLOT", "Funky Games"
]

PRODUCT_DETAIL = [
    "SABA Basketball", "SABA Basketball PinGoal", "SABA E-Sports PinGoal",
    "SABA Other Sports", "SABA Soccer", "SABA Soccer PinGoal", "SABA Tennis",
    "Sportsbook", "Allbet Promotion", "PP Jackpot Contribution", "PP Jackpot Prize",
    "SG Jackpot Contribution", "SG Jackpot Prize", "AE Sexy Lucky Draw",
    "Vgaming Promotion Prize", "MG-RNG Promotion Prize", "Live22 Promotion Prize",
    "FastSpin Promotion Prize", "FA CHAI Promotion Prize", "ASKMESLOT Promotion Prize"
]

LEVEL = ['Super Agent', 'Master Agent', 'Agent', 'Direct Member']

# customer = pd.read_csv(r'/home/hoangtv/Desktop/Long/llm-tuning/data/customer.csv')
customer = pd.read_csv(r'D:\\Desktop\\AlphaReportChatbot\\data\\customer.csv')
USER = customer.dropna()['username'].tolist()
    

In [2]:
def get_current_year() -> str:
    """
    Get the current year in YYYY format.
    
    Returns:
        str: Current year in YYYY format
    """
    from datetime import datetime
    return datetime.now().strftime("%Y")



def get_current_datetime() -> str:
    """
    Get the current datetime in YYYY-MM-DD format.
    
    Returns:
        str: Current date in YYYY-MM-DD format
    """
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")



def get_current_month() -> str:
    """
    Get the current month in MM format.
    
    Returns:
        str: Current month in MM format
    """
    from datetime import datetime
    return datetime.now().strftime("%m")

In [11]:
import requests
import json

current_date = get_current_datetime()
current_year = get_current_year()
current_month = get_current_month()

def extract_entity_by_ollama(
  query: str,
  url = 'https://saillm.oneops.net/api/chat', 
  model='qwen2.5:14b'
):
  
  system_prompt = "You are an AI assistant majoring for Named Entity Recognition trained to extract entity and categorize queries for Winlost Report Detail"
  
  user_prompt = f"""
        Current date: {current_date}
        Current year: {current_year}
        Current month: {current_month}

        Extract the most relevant keywords from the following sentence: '{query}'. 
        Focus on important nouns that convey the core meaning. 
        Detect any words related to dates such as tomorrow, today, last week, next year, so on, following the example below.
        Help me convert the date range to the format of YYYY-MM-DD to YYYY-MM-DD.
        For date range, please help me convert it to from_date and to_date in DD/MM/YYYY format following these rules:

        1. If a single date is mentioned (e.g. "day 10"):
           - Use current month and year {current_year} and {current_month}
           - Set both from_date and to_date to that date
           Example: "day 10" in March 2025 -> from_date: 10/03/2025, to_date: 10/03/2025

        2. If a date range is specified (e.g. "01/02/2024 to 15/02/2024"):
           - Keep the dates as specified in DD/MM/YYYY format
           Example: "01/02/2024 to 15/02/2024" -> from_date: 01/02/2024, to_date: 15/02/2024

        3. If relative dates are mentioned:
           - "today" -> Use {current_date} for both
           - "yesterday" -> Use yesterday's date for both from current date {current_date}
           - "last week" -> from_date is 7 days ago, to_date is today from current date {current_date}
           - "last month" -> from_date is 1st of previous month, to_date is last day of previous month from current date {current_date}
           - "last year" -> from_date is Jan 1st of previous year, to_date is Dec 31st of previous year from current date {current_date}
           - "this week" -> from_date is Monday of current week, to_date is today from current date {current_date}
           - "this month" -> from_date is 1st of current month, to_date is today from current date {current_date}
           - "this year" -> from_date is Jan 1st of current year, to_date is today from current date {current_date}
           
        4. If a month range is specified (e.g. "1/1 to 31/1"):
           - Use current year {current_year}
           - Set from_date to first day of specified month 
           - Set to_date to last day of specified month
           Example: "1/1 to 31/1" in 2025 -> from_date: 01/01/2025, to_date: 31/01/2025
           
        5. If no date is specified:
           - Set date_range as "N/A"
           - Set both from_date and to_date as "N/A"
           
        If no relevant keywords are detected, return 'All' (except for dates, you must fill 'N/A').
        If the date range is not specified, please return 'N/A' for date_range.
        If the product is not specified, please return 'All' for product.
        If the product detail is not specified, please return 'All' for product_detail.
        If the level is not specified, please return 'All' for level.
        If the user is not specified, please return 'N/A' for user.
        
        Here is the list of product and product detail you should detect:
        ### PRODUCT = {PRODUCT}
        ### PRODUCT_DETAIL = {PRODUCT_DETAIL}
        ### LEVEL = {LEVEL}
        
        Occasionaly, "user" keyword may appear some words such as "master*", "super*", "admin*", "user*", "agent*" where * is any characters.
        
        Example 1:
        ## User: Get me a Turnover Detail Report on day 10
        ## Output:
        {{
            "date_range": "day 10",
            "from_date": "10/03/2025",
            "to_date": "10/03/2025",
            "product": "All",
            "product_detail": "All",
            "level": "All",
            "user": "N/A"
        }}
        
        Example 2:
        ## User: Get me a Net Turnover Report for Direct Member who played Product Detail Sportsbook in Sportsbook Product from 01/02/2024 to 15/02/2024
        ## Output:
        {{
            "date_range": "01/02/2024 to 15/02/2024",
            "from_date": "01/02/2024",
            "to_date": "15/02/2024",
            "product": "Sportsbook",
            "product_detail": "Sportsbook",
            "level": "Direct Member",
            "user": "N/A"
        }}
        
        Example 3:
        ## User: Get me a Gross Commission Detail Report for Super Agent who played Product Detail SABA Basketball in SABA Basketball Product from 01/02/2024 to 15/02/2024
        ## Output:
        {{
            "date_range": "01/02/2024 to 15/02/2024",
            "from_date": "01/02/2024",
            "to_date": "15/02/2024",
            "product": "SABA Basketball",
            "product_detail": "SABA Basketball",
            "level": "Super Agent",
            "user": "N/A"
        }}
        
        Example 4:
        ## User: Bet Count Detail Report for Product Sportsbook
        ## Output:
        {{
            "date_range": "N/A",
            "from_date": "N/A",
            "to_date": "N/A",
            "product": "Sportsbook",
            "product_detail": "All",
            "level": "All",
            "user": "N/A"
        }}
        """


  headers = {
      "Content-Type": "application/json"
  }

  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_prompt}
  ]


  format={
      "type": "object",
      "properties": {
          "date_range": {"type": "string"},
          "from_date": {"type": "string"},
          "to_date": {"type": "string"},
          "product": {"type": "string"},
          "product_detail": {"type": "string"},
          "level": {"type": "string"},
          "user": {"type": "string"}
      },
      "required": ["function_called", "task", "date_range", "from_date", "to_date", "product", "product_detail", "level", "user"]
  }

  data = {
      "model": model,
      "messages": messages,
      "format": format,
      "stream": False
  }

  response = requests.post(url, headers=headers, data=json.dumps(data))

  # if not response.status_code == 200:
  #     print("Invalid JSON response:", response.text)
  # else:
  #     print(f"Error: {response.status_code}, {response.text}")
      
  entities = json.loads(response.json()['message']['content'])
  
  return entities


In [12]:
from spacy import displacy
import time
from typing import List

def convert_to_spacy_format(entities_dict, query):
    spacy_entities = []
    for label, text in entities_dict.items():
        if text != "All" and text != "N/A":
            start = query.find(text)
            if start != -1:
                end = start + len(text)
                spacy_entities.append({
                    "start": start,
                    "end": end,
                    "text": text,
                    "label": label,
                    "score": 0.99  # Placeholder score, adjust as needed
                })
    return spacy_entities

def test(
    queries: List[str]
) -> None:
    for query in queries:
        
        start = time.time()
        entities = extract_entity_by_ollama(
            query=query,
            url = 'https://ollama.selab.edu.vn/api/chat',
            model='qwen2.5:14b'
        )
        print(entities)
        entities = convert_to_spacy_format(entities, query)
        dic_ents = {
            "text": query,
            "ents": entities,
            "title": None
        }
        
        print(f"🫂 QUERY: {query}")
        print(f"😍 ENTITIES: ")
        displacy.render(dic_ents, manual=True, style='ent')
        print(f"TIME INFERENCE: {time.time() - start}")
        print("🤖==========================================================================================🤖")
        print("\n")
    

In [13]:
queries = [
    "Get me a Win Loss Detail Report yesterday",
    "Get me a Turnover Detail Report on day 10",
    "Net Turnover Detail Report from 1/1 to 31/1",
    "Get me a Win Loss Detail Report for Direct Member who played Product Detail Sportsbook in Sportsbook Product from 01/02/2024 to 15/02/2024",
    "Give me my Win Loss Detail report  last week",
    "Give me a Win Loss Detail report for Sportsbook from last week.",
    "Give me my Win Loss Detail report  last week",
    "Get me a Company Report of master1",
    "Win/Loss details for Product Sportsbook",
    "Get me a Win Loss on day 10",
    "I want Win Loss report of Sportsbook this week",
    "I want performance of abc1 last week",
    "Live Casino Win Loss report of Master master1 yesterday",
    "I need Win Loss report of Agent agent01 last week",
    "Show me today's summary",
    "How did we perform in live casino last week?",
    "The amount I need to pay for master abc1 last week",
    "Get report of master1",
    "Win/Loss details for Product Sportsbook",
    "Get me a Win Loss Detail Report of master1"
]

test(queries=queries)

{'date_range': 'yesterday', 'from_date': '02/04/2025', 'to_date': '02/04/2025', 'product': 'All', 'product_detail': 'All', 'level': 'All', 'user': 'N/A'}
🫂 QUERY: Get me a Win Loss Detail Report yesterday
😍 ENTITIES: 


TIME INFERENCE: 5.138331413269043
🤖==========================================================================================🤖


{'date_range': 'day 10', 'from_date': '10/04/2025', 'to_date': '10/04/2025', 'product': 'All', 'product_detail': 'All', 'level': 'All', 'user': 'N/A'}
🫂 QUERY: Get me a Turnover Detail Report on day 10
😍 ENTITIES: 


TIME INFERENCE: 5.034808874130249
🤖==========================================================================================🤖


{'date_range': '1/1 to 31/1', 'from_date': '01/01/2025', 'to_date': '31/01/2025', 'product': 'All', 'product_detail': 'All', 'level': 'All', 'user': 'N/A'}
🫂 QUERY: Net Turnover Detail Report from 1/1 to 31/1
😍 ENTITIES: 


TIME INFERENCE: 5.2050886154174805
🤖==========================================================================================🤖


{'date_range': '01/02/2024 to 15/02/2024', 'from_date': '01/02/2024', 'to_date': '15/02/2024', 'product': 'Sportsbook', 'product_detail': 'Sportsbook', 'level': 'Direct Member', 'user': 'N/A'}
🫂 QUERY: Get me a Win Loss Detail Report for Direct Member who played Product Detail Sportsbook in Sportsbook Product from 01/02/2024 to 15/02/2024
😍 ENTITIES: 


TIME INFERENCE: 5.862730979919434
🤖==========================================================================================🤖


{'date_range': 'last week', 'from_date': '27/03/2025', 'to_date': '02/04/2025', 'product': 'All', 'product_detail': 'All', 'level': 'All', 'user': 'N/A'}
🫂 QUERY: Give me my Win Loss Detail report  last week
😍 ENTITIES: 


...

TIME INFERENCE: 4.916509628295898
🤖==========================================================================================🤖


{'date_range': 'last week', 'from_date': '27/03/2025', 'to_date': '02/04/2025', 'product': 'All', 'product_detail': 'All', 'level': 'Master Agent', 'user': 'master abc1'}
🫂 QUERY: The amount I need to pay for master abc1 last week
😍 ENTITIES: 


TIME INFERENCE: 5.061266899108887
🤖==========================================================================================🤖


{'date_range': 'N/A', 'from_date': 'N/A', 'to_date': 'N/A', 'product': 'All', 'product_detail': 'All', 'level': 'Master Agent', 'user': 'master1'}
🫂 QUERY: Get report of master1
😍 ENTITIES: 


TIME INFERENCE: 4.570035696029663
🤖==========================================================================================🤖


{'date_range': 'N/A', 'from_date': 'N/A', 'to_date': 'N/A', 'product': 'Sportsbook', 'product_detail': 'All', 'level': 'All', 'user': 'N/A'}
🫂 QUERY: Win/Loss details for Product Sportsbook
😍 ENTITIES: 


TIME INFERENCE: 4.615182876586914
🤖==========================================================================================🤖


{'date_range': 'N/A', 'from_date': 'N/A', 'to_date': 'N/A', 'product': 'All', 'product_detail': 'All', 'level': 'Master Agent', 'user': 'master1'}
🫂 QUERY: Get me a Win Loss Detail Report of master1
😍 ENTITIES: 


TIME INFERENCE: 4.564401388168335
🤖==========================================================================================🤖




In [13]:
import json

with open(r'D:\Desktop\AlphaReportChatbot\src\notebook\research\correct_arb_datasets.json', 'r') as file:
    data = json.load(file)

queries2 = [t['query'] for t in data]

test(queries=queries2)

{'date_range': '10/05/2023 to 15/06/2023', 'from_date': '10/05/2023', 'to_date': '15/06/2023', 'product': 'Sportsbook', 'product_detail': 'All', 'level': 'Agent', 'user': 'ABCTeam2345'}
🫂 QUERY: Can you generate a Win/Loss report for ABCTeam, specifically for the user ABCTeam2345, who is an active member within the Sportsbook Product with details during the period from 10/05/2023 to 15/06/2023?
😍 ENTITIES: 


TIME INFERENCE: 11.69409728050232
🤖==========================================================================================🤖


{'date_range': 'February to April 2023', 'from_date': '01/02/2023', 'to_date': '30/04/2023', 'product': 'Sportsbook', 'product_detail': 'All', 'level': 'All', 'user': 'ABCTeam2345'}
🫂 QUERY: I need a Win/Loss report for the user ABCTeam2345 in the Sportsbook Product covering activities from February to April 2023.
😍 ENTITIES: 


TIME INFERENCE: 28.025195598602295
🤖==========================================================================================🤖


{'date_range': 'March 20 to April 30, 2023', 'from_date': '20/03/2023', 'to_date': '30/04/2023', 'product': 'Sportsbook', 'product_detail': 'All', 'level': 'All', 'user': 'ABCTeam'}
🫂 QUERY: Please provide a Win/Loss report for the user ABCTeam in the Sportsbook Product with details between the dates of March 20 and April 30, 2023.
😍 ENTITIES: 


TIME INFERENCE: 6.829957962036133
🤖==========================================================================================🤖


{'date_range': 'January', 'from_date': '01/01/2025', 'to_date': '31/01/2025', 'product': 'Sportsbook', 'product_detail': 'All', 'level': 'Direct Member', 'user': 'AB2C3D4E5'}
🫂 QUERY: Could you provide a detailed Win/Loss report for AB2C3D4E5, who is an active member under the Sportsbook product with details of transactions during January?
😍 ENTITIES: 


TIME INFERENCE: 6.309334754943848
🤖==========================================================================================🤖


{'date_range': 'February', 'from_date': '01/02/2025', 'to_date': '29/02/2025', 'product': 'Sportsbook', 'product_detail': 'All', 'level': 'All', 'user': 'AB2C3D4E5'}
🫂 QUERY: Please generate a report for user AB2C3D4E5 indicating their activity in the Sportsbook product during February.
😍 ENTITIES: 


...

TIME INFERENCE: 5.157460927963257
🤖==========================================================================================🤖


{'date_range': 'April 20th to May 30th', 'from_date': '20/04/2025', 'to_date': '30/05/2025', 'product': 'Sportsbook', 'product_detail': 'All', 'level': 'All', 'user': 'B567'}
🫂 QUERY: I need the performance summary of user B567 in the Sportsbook Product from April 20th to May 30th.
😍 ENTITIES: 


TIME INFERENCE: 5.156300067901611
🤖==========================================================================================🤖


{'date_range': 'June 5th to July 1st', 'from_date': '05/06/2025', 'to_date': '01/07/2025', 'product': 'Sportsbook', 'product_detail': 'All', 'level': 'All', 'user': 'C890'}
🫂 QUERY: Generate a report for user C890's performance in the Sportsbook Product between June 5th and July 1st.
😍 ENTITIES: 


TIME INFERENCE: 5.057621240615845
🤖==========================================================================================🤖


{'date_range': 'July 15th to August 20th', 'from_date': '15/07/2023', 'to_date': '20/08/2023', 'product': 'Sportsbook', 'product_detail': 'All', 'level': 'All', 'user': 'D123'}
🫂 QUERY: Provide a detailed performance report for user D123 in the Sportsbook Product covering July 15th to August 20th.
😍 ENTITIES: 


TIME INFERENCE: 5.158004999160767
🤖==========================================================================================🤖


{'date_range': 'September 10th to October 1st', 'from_date': '10/09/2025', 'to_date': '01/10/2025', 'product': 'Sportsbook', 'product_detail': 'All', 'level': 'All', 'user': 'E456'}
🫂 QUERY: Could you compile a performance report for user E456 in the Sportsbook Product from September 10th to October 1st?
😍 ENTITIES: 


TIME INFERENCE: 5.095315456390381
🤖==========================================================================================🤖


